In [3]:
import os
# os.system('cls')  # On Windows System

from IPython import get_ipython
get_ipython().run_line_magic('reset','-sf')

In [4]:
import numpy as np # Import numpy
import matplotlib.pyplot as plt # Import pyplot
import pandas as pd
from sklearn.model_selection import train_test_split
# plt.close('all')

In [5]:
# import the features and the labels for the training data and the test data
# import the training data
# train_data = pd.read_csv('train.csv')
# test_data = pd.read_csv('test.csv')

train_data = np.loadtxt('train.csv', delimiter=',', skiprows=1) 
test_data = np.loadtxt('test.csv', delimiter=',', skiprows=1)

train_data = train_data[:800]
test_data = test_data[:200]

In [6]:
split_index = int(0.5 * len(test_data))


In [7]:
eval_data = test_data[:split_index]
test_data = test_data[split_index:]

In [8]:
eval_data.shape

(100, 11)

In [9]:
test_data.shape

(100, 11)

In [10]:
train_data.shape

(800, 11)

In [11]:
# normalize the training data using z-normalization 
mean = np.mean(train_data, axis=0)
std = np.std(train_data, axis=0)
normalized_train_data = (train_data - mean)/std

In [12]:
# normalize the training data using z-normalization but using the training data mean and standard deviation
# normalize the data using z-normalization 
normalized_test_data = (test_data - mean)/std

In [13]:
# sigmoid() function (activation function)
def sigmoid(x):
    fAct = 1/(1+np.exp(-1*x))
    return fAct


In [14]:
# Derivation of the activation fucntion
def derived_sigmoid(x):
    fAct = 1/(1+np.exp(-1*x))
    DfAct = fAct*(1-fAct)
    return DfAct

In [15]:
# Generate m*n matrix, initial values are zeros
def makematrix(m, n, fill=0.0):
    a = []
    for i in range(m):
        a.append([fill]*n)
    return np.array(a)

In [16]:
# Random data
def random_number(a,b):
    return (b-a)*np.random.normal()+a

In [17]:
normalized_train_data.shape

(800, 11)

In [18]:
len(normalized_test_data[:,0])

100

In [27]:
#BP Neural Networks
class BPNN:
    def __init__(self, num_in, num_hidden, num_out):
        # Nodes of input, hidden, output layers
        self.num_in = num_in + 1  # Add one bias node
        self.num_hidden = num_hidden + 1   # Add one bias node
        # self.num1_hidden = num1_hidden + 1
        self.num_out = num_out
        
        # Activate all nodes (vector)
        self.active_in = np.array([1.0]*self.num_in)
        self.active_hidden = np.array([1.0]*self.num_hidden)
        # self.active_hidden1 = np.array([1.0]*self.num1_hidden)
        self.active_out = np.array([1.0]*self.num_out)

        # load the weights
        # Load the weights from the file
        loaded_weights = np.load('weights.npy', allow_pickle=True) 
        print("Loading data from the weights")
        print(loaded_weights)
        
        # Create weight matrices
        self.wight_in = makematrix(self.num_in, self.num_hidden-1)
        # self.wight_in1 = makematrix(self.num_hidden, self.num1_hidden-1)
        self.wight_out = makematrix(self.num_hidden, self.num_out)
        
        
              
    # Feed-forward
    def Feedforward(self, inputs):
        if np.shape(inputs)[1] != self.num_in-1:
            raise ValueError('Incorrect input numbers')
        # Input layer values
        self.active_in[1:self.num_in]=inputs
        
        # Hidden layer values
        self.sum_hidden=np.dot(self.wight_in.T,np.array([self.active_in]).T)
        self.active_hidden = np.vstack( (1, sigmoid(self.sum_hidden)) )   # Activation function
        
        # Output layer values
        self.sum_out=np.dot(self.wight_out.T,self.active_hidden)
        self.active_out = self.sum_out # or sigmoid(self.sum_out)
        return self.active_out
 
    # Backpropagation
    def errorbackpropagate(self, targets):   # lr is the learning rate
        if self.num_out==1:
            targets=targets
        if np.shape(targets)[1] != self.num_out:
            raise ValueError('Incorrect Output numbers')
        # Cost function
        self.error=(1/2)*np.dot((self.active_out-targets.T).T,(self.active_out-targets.T))
        
        # dJ/dw_out
        self.gradient_out = (self.active_out-targets.T) #*derived_sigmoid(self.sum_out)#dJ/dx
        self.gradient_w_out = np.dot(self.gradient_out,self.active_hidden.T).T #dx/dw_out
        
        # dJ/dw_in
        self.gradient_hidden = np.dot(self.wight_out[1:],self.gradient_out)*derived_sigmoid(self.sum_hidden) #dJ/dx
        self.gradient_w_in = np.dot(self.gradient_hidden,np.array([self.active_in])).T #dx/dw_in
       
        return self.error

    def train(self, pattern, itera=1000, lr = 0.05):
        error = 1000
        i=0
        # while(error < 185):
        for i in range(itera):
            Gradient_out = 0.0
            Gradient_in = 0.0
            error = 0
            i+=1
            for j in pattern:
                inputs = np.array([j[0:self.num_in-1]])
                
                targets = np.array([j[self.num_in-1:]])
                self.Feedforward(inputs)
                self.errorbackpropagate(targets)
                Gradient_out = Gradient_out + self.gradient_w_out
                Gradient_in = Gradient_in + self.gradient_w_in
            
                error = error + self.error
                
            [Tm,Tn] = np.shape(pattern)
            self.wight_out = self.wight_out - lr*Gradient_out/Tm
            self.wight_in = self.wight_in - lr*Gradient_in/Tm 
            
            # if i % 10 == 0:
            #     print('################# error %-.5f##################The %d th iteration' %(error,i))
    # Weights
    def weights(self):
        print("input weights")
        print(self.wight_in)
        print("Output weights")
        print(self.wight_out)
        
    # Test prediction
    def test(self, patterns):
        Pre = []
        for i in patterns:
            inputs = np.array([i[0:self.num_in-1]])
            Pre.append( self.Feedforward(inputs) )
            # print(Pre)
        return Pre         


n = BPNN(10, 16, 1) #Create neural networK  
# n.train(normalized_train_data) # Training BP
     
# d=n.test(normalized_test_data)
# len(d)
# y_pred = np.zeros((len(normalized_test_data[:,0]),))
# for i in range(len(d)):
#   y_pred[i] = d[i][0,0]
    
# n.weights() # check weights
# np.save('weights.npy', n.weights())


Loading data from the weights
None


In [ ]:
187.65572